In [1]:
import numpy as np
import sys
import itertools as itertools
sys.path.append("..")
sys.path.append("../..")

import matplotlib.pyplot as plt
from python.Dzyalonshinskii_Moriya_Ising import DM_Ising
from python.DMRG import DMRG
from python.Zippers import MPO_to_Hamiltonian, contract_MPS
from python.Canonical_Form import get_Neumann_entropy
from python.Backend import Backend
from python.Decomposition import EIGH

In [32]:
n_sites = 10

J = 1.0
D = 2.0
magnetic_field = 0.0

bk = Backend("numpy")
# bk = Backend("torch")

Hamiltonian = DM_Ising(
    n_sites=n_sites, J=J, D=D, magnetic_field=magnetic_field, bk=bk
)

MPO_transposed = [bk.transpose(ham, (2, 3, 0, 1)) for ham in Hamiltonian]

In [33]:
"""
Get exact matrix (exponential barrier)
"""

matrix = MPO_to_Hamiltonian(MPO_transposed, bk=bk)
eigvals, _ = EIGH(matrix, bk=bk)
eigvals

array([-7.18050141, -6.71853006, -6.71853006, ...,  4.96634056,
        4.96634056,  5.09340351])

In [34]:
bk.norm(matrix - matrix.conj().T)

0.0

In [35]:
NKeep = 20
NSweep = 10
Krylov_bases = 5 #* 5 is usually enough
Lanczos_cutoff = 1e-2
iterative_diag = True
two_site = True
verbose = True

In [36]:
ground_energies, ground_times, ground_MPS = DMRG(
    bk = bk,
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = iterative_diag,
    two_site = two_site,
    verbose = verbose,
)

eigvals[0]

L=10 | NKeep=20 | NSweep=10 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=-7.1778373 | time=0js
iter=1 | energy=(-7.180501091285148+0j) | time=0.21487506999983452s
iter=2 | energy=(-7.18050109128515+0j) | time=0.3400266589997045s


-7.180501405973142

In [37]:
excited_energies, excited_times, excited_MPS = DMRG(
    bk = bk,
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = False,
    orthogonal_to_list_of_MPS=[ground_MPS],
    two_site = two_site,
    verbose = verbose,
)

eigvals[1]

L=10 | NKeep=20 | NSweep=10 | diag=False | two=True | Krylov=5 | cutoff=0.01
Random initialization complete
iter=0 | energy=-0.064854027 | time=0js
iter=1 | energy=(-6.7184792750981135+0j) | time=0.3177847280003334s
iter=2 | energy=(-6.718516606075855+0j) | time=0.564999849000742s
iter=3 | energy=(-6.718516606075852+0j) | time=0.9215987240013419s


-6.718530060700819

In [38]:
get_Neumann_entropy(ground_MPS, bk=bk)

array([1.        +0.j, 0.62279999+0.j, 1.05247991+0.j, 0.75064355+0.j,
       1.07100508+0.j, 0.75074456+0.j, 1.05249544+0.j, 0.6228611 +0.j,
       1.        +0.j])

In [39]:
get_Neumann_entropy(excited_MPS, bk=bk)

array([0.90598768+0.j, 0.90134731+0.j, 0.95163355+0.j, 1.26081054+0.j,
       0.98804787+0.j, 1.26089317+0.j, 0.9517061 +0.j, 0.90185461+0.j,
       0.90587701+0.j])